# 🤺 Demo: Agents and MCP Servers

## Demo overview
In this demo notebook, we will explore how to create and interact with an MCP server using the `mcp` Python package and `fence-llm`. 
- First section; Setup a simple Local MCP server (via the `Agent` class and the `BedrockAgent` class)
- Second section; Interact with a third-party MCP server

## Model Context Protocal

The Model Context Protocol (MCP) is an open standard introduced by Anthropic in November 2024 to streamline the integration of AI models, particularly large language models (LLMs), with external tools, systems, and data sources. Often likened to the “USB-C for AI apps,” MCP provides a universal interface that allows AI assistants to interact with various software components in a consistent manner .

An MCP server acts as a bridge between AI agents and external functionalities. It exposes tools, enabling AI models to perform actions like reading files, querying databases, or invoking APIs without the need for custom integrations. This modular approach promotes interoperability and simplifies the development of AI-driven applications.

## Transport options
Transport Options for MCP Servers

MCP supports multiple transport mechanisms to accommodate different deployment scenarios:
- __Standard Input/Output (stdio)__: Ideal for local development and testing, stdio allows MCP servers to communicate through the standard input and output streams. This method is straightforward to implement and is well-suited for environments where simplicity and ease of setup are priorities .
- __Server-Sent Events (SSE)__: SSE enables unidirectional, real-time communication from the server to the client over HTTP. It’s useful for scenarios where the server needs to push updates to the client without requiring complex bidirectional protocols. However, as of March 2025, SSE has been deprecated in favor of more robust solutions . ￼
- __HTTP Stream Transport__: This modern transport mechanism supports bidirectional communication and streaming over HTTP, providing a more flexible and scalable solution for web-based MCP applications. It allows for advanced features like session management and comprehensive authentication options .


**Note** This notebook assumes you have access to AWS Bedrock, as we use Bedrock's Claude-instant model to fuel our LLM interactions.

In [39]:
from pathlib import Path
current_dir = Path('.').resolve().parents[1]  # now goes two levels up
import sys
sys.path.append(str(current_dir))

## Local MCP Server

In [ ]:
from mcp import StdioServerParameters
from mcp.client.stdio import stdio_client

from fence.tools.mcp.mcp_client import MCPClient
from fence.agents.bedrock import BedrockAgent
from fence.models.bedrock import NovaPro

from fence.utils.logger import setup_logging
# Setup logging
logger = setup_logging(__name__, log_level="kill")  # On

In [42]:
stdio_mcp_client = MCPClient(lambda: stdio_client(
    StdioServerParameters(
        command="uvx", 
        args=["awslabs.aws-documentation-mcp-server@latest"]
    )
))

In [43]:
# Create an Bedrock agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    tools = stdio_mcp_client.list_tools_sync()
    
    agent = BedrockAgent(
        model=NovaPro(cross_region='eu'),
        tools=tools,
    )
    
    response = agent.run("What is the longest time an AWS lambda method can run?")
    

BedrockAgent 🚀 [start] Agent BedrockAgent started
BedrockAgent 💭 [thought] To find the longest time an AWS Lambda method can run, I need to search the AWS documentation for relevant information. I'll start by using the `search_documentation` tool to find the maximum execution time for AWS Lambda functions.
BedrockAgent 🔧 [tool_use] Tool [search_documentation] requested with parameters: {'limit': 10, 'search_phrase': 'AWS Lambda maximum execution time'}
BedrockAgent 🔧 [tool_use] Tool [search_documentation] executed with parameters: {'limit': 10, 'search_phrase': 'AWS Lambda maximum execution time'} -> {"content": [{"text": "{\n  \"rank_order\": 1,\n  \"url\": \"https://docs.aws.amazon.com/lambda/latest/dg/lambda-intro-execution-role.html\",\n  \"title\": \"Defining Lambda function permissions with an execution role - AWS Lambda\",\n  \"context\": \"Learn how to manage permissions for your Lambda function's execution role.\"\n}"}, {"text": "{\n  \"rank_order\": 2,\n  \"url\": \"https://d

## Using Streaming HTTP Transport

In [ ]:

from mcp.client.streamable_http import streamablehttp_client
from fence.tools.mcp.mcp_client import MCPClient

from fence.agents.bedrock import BedrockAgent
from fence.models.bedrock import NovaPro

from fence.agents.agent import Agent
from fence.models.openai import GPT4o


# for the http client we'll use the streamablehttp_client with a local MCP server intearcting with Github

# git clone https://github.com/smithery-ai/mcp-servers.git
# cd mcp-servers/github
# npm run start
# This with start a local MCP server on port 8081

http_mcp_client = MCPClient(lambda: streamablehttp_client("http://localhost:8081/mcp"))
http_mcp_client.start()

tools = http_mcp_client.list_tools_sync()

In [49]:
agent = BedrockAgent(
        model=NovaPro(cross_region='eu'),
        tools=tools,
    )

In [50]:
answer  = agent.run("Tell me about the fence project by WouterDurnez")

BedrockAgent 🚀 [start] Agent BedrockAgent started
BedrockAgent 💭 [thought] To gather information about the "fence project" by WouterDurnez, I should first search for repositories related to this project. If I find relevant repositories, I can then get detailed information about them.
BedrockAgent 🔧 [tool_use] Tool [search_repositories] requested with parameters: {'query': 'fence project user:WouterDurnez'}
BedrockAgent 🔧 [tool_use] Tool [search_repositories] executed with parameters: {'query': 'fence project user:WouterDurnez'} -> No repositories found
BedrockAgent 💭 [thought] Since no repositories were found under the query "fence project user:WouterDurnez", I should try a broader search to see if there are any repositories by WouterDurnez that might be related to a fence project.
BedrockAgent 🔧 [tool_use] Tool [search_repositories] requested with parameters: {'query': 'user:WouterDurnez'}
BedrockAgent 🔧 [tool_use] Tool [search_repositories] executed with parameters: {'query': 'user:W

In [59]:
agent = BedrockAgent(
        model=NovaPro(cross_region='eu'),
        tools=tools,
    )

In [60]:
agent.run("Tell me about the fence project by WouterDurnez")

BedrockAgent 🚀 [start] Agent BedrockAgent started
BedrockAgent 💭 [thought] To gather information about the fence project by WouterDurnez, I should first search for repositories related to this project. I'll use the `search_repositories` tool with a query that includes the user's name and the project name.
BedrockAgent 🔧 [tool_use] Tool [search_repositories] requested with parameters: {'query': 'user:WouterDurnez fence'}
BedrockAgent 🔧 [tool_use] Tool [search_repositories] executed with parameters: {'query': 'user:WouterDurnez fence'} -> ### Found 2 repositories:

1. **WouterDurnez/fence** - 13 stars - `Python`
   A small package that offers an alternative for LangChain's most used functionality
   _Updated: 2025-05-27_

2. **WouterDurnez/fenced-playground**
   Various snippets of functionality using Fence
   _Updated: 2024-12-03_
BedrockAgent 🎯 [answer] The fence project by WouterDurnez consists of two repositories:

1. **WouterDurnez/fence**:
   - Stars: 13
   - Language: Python
   - 

AgentResponse(answer="The fence project by WouterDurnez consists of two repositories:\n\n1. **WouterDurnez/fence**:\n   - Stars: 13\n   - Language: Python\n   - Description: A small package that offers an alternative for LangChain's most used functionality.\n   - Last updated: 2025-05-27\n\n2. **WouterDurnez/fenced-playground**:\n   - Description: Various snippets of functionality using Fence.\n   - Last updated: 2024-12-03\n\nWould you like more detailed information about either of these repositories?", events=[BedrockAgent [Start], BedrockAgent [Thinking] To gather information about the fence project by WouterDurnez, I should first search for repositories related to this project. I'll use the `search_repositories` tool with a query that includes the user's name and the project name., BedrockAgent [Tool_use_start] search_repositories ({'query': 'user:WouterDurnez fence'}) -> None, BedrockAgent [Tool_use_stop] search_repositories ({'query': 'user:WouterDurnez fence'}) -> ### Found 2 re